<a href="https://colab.research.google.com/github/ericr6/platform_analysis/blob/main/TRD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task analysis of iExec MArketplace
This notebook create dataframe using the graph,then from the dataframe you can easily create dashboard or explore data.

Section 0 : import and functions needed

Section 1 : retrieve latest data, latest N days with parameter N

Section 3 : Visualize recent TRD

Section 4 : Save historical data in google drive [DO NOT USE]

Section 5 : Load full historical data from google drive

Section 6 : Visualize full historical TRD

## Section 0 : import and functions needed


In [1]:
import requests
import json
import csv
from datetime import datetime, timedelta
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# @title Common Functions
# Function to convert Hexa String to Text
def hex_to_string(value):
    if not value:
        return value

    bytes_from_hex = bytes.fromhex(value[2:])
    return bytes_from_hex.decode('utf-8')

def filter_by_time_interval(data_list, date_begin, date_end):
    # Convert date_begin and date_end strings to datetime objects
    #date_begin = datetime.strptime(date_begin, '%Y-%m-%d %H:%M:%S')
    #date_end = datetime.strptime(date_end, '%Y-%m-%d %H:%M:%S')

    # Filter data_list based on timestamp falling within the specified time interval
    filtered_data = [row for row in data_list if date_begin <= datetime.utcfromtimestamp(int(row[0])) <= date_end]
    return filtered_data



def get_api_data_iexec_interval(url_query, query, global_skip, date_begin=None, date_end=None):
    total_data = []
    are_data = True
    detected = False
    skip = 0
    while are_data or not detected:

        # Construct the query with skip parameter and optional date filters
        query_iter = query.replace("skip_param", str(global_skip + skip))

        options = {
            'headers': {'Content-Type': 'application/json'},
            'data': '{"query": "' + query_iter + '"}',
        }

        response = requests.post(url_query, **options)

        # Check if the response contains valid JSON data
        response.raise_for_status()
        data = response.json()

        array_data = data.get('data', {}).get('deals', [])

        grouped = [
            [
                e['timestamp'],
                e['workerpoolPrice'],
                e['datasetPrice'],
                e['appPrice'],
                e['app']['name'],
                hex_to_string(e['app']['multiaddr']),
                e['workerpool']['id'],
            ]
            for e in array_data
        ]
        grouped = filter_by_time_interval(grouped, date_begin, date_end)

        total_data.extend(grouped)
        skip += 500
        _are_data = len(grouped) > 0
        if _are_data > 0 and are_data:
 #           print ("detected is true")
            detected = True
        are_data = len(grouped) > 0
        print("i= " + str(global_skip + skip) + " are_data: " + str(are_data) + "_are_data: " + str(_are_data) + " grouped size " +  str(len(grouped)) + " array_data size: " + str(len(array_data))  )
    return total_data



# Actualize data
url_query_bellecour = 'https://thegraph.bellecour.iex.ec/subgraphs/name/bellecour/poco-v5'
query = '{\
                deals(first:500, skip: skip_param, orderBy: timestamp, orderDirection: desc){\
                    timestamp\
                    workerpoolPrice\
                    datasetPrice\
                    appPrice\
                    app{\
                      name\
                      multiaddr,\
                    }\
                    workerpool{\
                      id\
                    }\
                  }\
                }'





ModuleNotFoundError: No module named 'pandas'

## Section 1 : retrieve latest data, default (30 days)


In [ ]:
# @title Get latest data ; Select N Days
_nbday=30
date_end = datetime.now()
date_begin = date_end - timedelta(days=_nbday)

print ("Period to retrieve:", date_begin, date_end)

#Define the desired data types
dtypes = {
    'DATE': 'object',
    'workerpoolPrice': 'float64',
    'datasetPrice': 'float64',
    'appPrice': 'float64',
    'appName': 'object',
    'appAddress': 'object',
    'workerpool': 'object'
}

api_data = get_api_data_iexec_interval(url_query_bellecour, query, 0, date_begin, date_end)
df_latest = pd.DataFrame(api_data, columns=["DATE", "workerpoolPrice", "datasetPrice","appPrice", "appName","appAddress", "workerpool"]).astype(dtypes)
df_latest["DATE"] = [datetime.utcfromtimestamp(int(x)) for x in df_latest["DATE"]]

print("Period observed", df_latest["DATE"].min(), df_latest["DATE"].max())
print("dataset length: " + str(df_latest.shape[0]))

Period to retrieve: 2024-02-17 07:42:18.105510 2024-03-18 07:42:18.105510
i= 500 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 1000 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 1500 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 2000 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 2500 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 3000 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 3500 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 4000 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 4500 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 5000 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 5500 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 6000 are_data: True_are_data: True grouped size 500 array_data size: 500
i= 6500 are_dat

Check dataset structure and data header, you can share to chatgpt this data structure, and ask him to plot anything you want, he will propose the code, then create a computing box and insert the code directly here to obtain what you ask

In [ ]:
# @title check data set structure

df_latest.dtypes


TASK_ID                  object
APP NAME                 object
APP MULTIADDR            object
TAG                      object
STATUS                   object
DATE             datetime64[ns]
WORKERPOOL ID            object
REQUESTER ID             object
dtype: object

In [ ]:
# @title check dataset header

df_latest

,TASK_ID,APP NAME,APP MULTIADDR,TAG,STATUS,DATE,WORKERPOOL ID,REQUESTER ID
0,0xe345b2fc524718e3e940d659dafec774a87620b9f3e5...,tee-python-hello-world:8.0.3-gramine,docker.io/iexechub/tee-python-hello-world:8.0....,0x00000000000000000000000000000000000000000000...,ACTIVE,2024-03-18 07:42:05,0xca76ab007f8a3fb64076851bb01acc99fc26d27b,0x5d62abd17f0a7f1004c981d5cff24c5a88d849f5
1,0xc53a373bd6b8555657e6237196c81106c713e1c4fb88...,python-hello-world:8.0.0-sconify-5.7.5-v14-pro...,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-03-18 07:36:15,0x0e7bc972c99187c191a17f3cae4a2711a4188c3f,0x6d6b5e88714263bbc6b4fa21a7328a7b947a302c
2,0xd62f49350db4a77ce3203fb2bf1e238d015410b4269f...,python-hello-world:8.0.0-sconify-5.7.5-v14-debug,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-03-18 07:34:30,0xdb214a4a444d176e22030be1ed89da1b029320f2,0x7d695b9a362e15090ce26711808bc89c18efa70f
3,0x47b40d3b40e84a72fda63c8a1fddaa1ac0d5d8f603d0...,python-hello-world:8.0.0-sconify-5.7.6-v15-pro...,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-03-18 07:26:30,0xca76ab007f8a3fb64076851bb01acc99fc26d27b,0x3d60589cb7a862e3001b8e7eff1ec248d1f16a18
4,0xeb15f32de5346781efd4ba7efd773a1394b18ed6637e...,python-hello-world:8.0.0-sconify-5.7.6-v15-debug,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-03-18 07:24:35,0xcfc79bf7ce221c504e281b42b4f50b1a0e692cb6,0x4d67487894140752d1a0dc81e77a578796e47255
...,...,...,...,...,...,...,...,...
11891,0x92f342d9cf93e523b65085e733ed4e483d71d258bd23...,python-hello-world:8.0.0-sconify-5.7.5-v14-pro...,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-02-17 07:57:05,0x0e7bc972c99187c191a17f3cae4a2711a4188c3f,0x6d6b5e88714263bbc6b4fa21a7328a7b947a302c
11892,0x23d8db34be141051fc23cfd0e8902effda5f2594c1c6...,python-hello-world:8.0.0-sconify-5.7.5-v14-debug,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-02-17 07:55:15,0xdb214a4a444d176e22030be1ed89da1b029320f2,0x7d695b9a362e15090ce26711808bc89c18efa70f
11893,0x4302a2bbbffa1a1bc419e8f945c4020af25ce9d4b7ce...,tee-python-hello-world:8.0.3-gramine,docker.io/iexechub/tee-python-hello-world:8.0....,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-02-17 07:54:05,0x0e7bc972c99187c191a17f3cae4a2711a4188c3f,0x8d6f890ea5d10134cd6b9777457eb75e76312304
11894,0x9b1e9d671d4da11f9c81de70774be7ae500cbdecce52...,python-hello-world:8.0.0-sconify-5.7.5-v14-debug,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,ACTIVE,2024-02-17 07:44:40,0xcfc79bf7ce221c504e281b42b4f50b1a0e692cb6,0x4d67487894140752d1a0dc81e77a578796e47255


## Section 2 : Visualize recent activity

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime


import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

def plot_price_histogram(df, period='daily'):
    """
    Create a histogram showing price sums by different time periods
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing DATE, workerpoolPrice, datasetPrice, and appPrice columns
    period (str): Aggregation period - 'daily', 'weekly', or 'monthly'
    
    Returns:
    pandas.DataFrame: Aggregated price data
    """
    # Ensure DATE is datetime
    df['DATE'] = pd.to_datetime(df['DATE'])
    
    # Create period groupings
    if period == 'daily':
        df['period'] = df['DATE'].dt.date
        date_format = '%Y-%m-%d'
    elif period == 'weekly':
        df['period'] = df['DATE'].dt.to_period('W').astype(str)
        date_format = 'Week %V, %Y'
    elif period == 'monthly':
        df['period'] = df['DATE'].dt.to_period('M').astype(str)
        date_format = '%B %Y'
    else:
        raise ValueError("Period must be 'daily', 'weekly', or 'monthly'")
    
    # Group by period and sum the prices
    period_sums = df.groupby('period').agg({
        'workerpoolPrice': 'sum',
        'datasetPrice': 'sum',
        'appPrice': 'sum'
    }).reset_index()
    
    if period == 'monthly':
      plt.figure(figsize=(20, 10))
    elif period == 'daily':
      plt.figure(figsize=(32, 10))
    elif period == 'weekly':
      plt.figure(figsize=(20, 10))


    # Plot each price type as a different colored bar
    bar_width = 0.25
    indices = range(len(period_sums['period']))
    
    bars1 = plt.bar(indices, period_sums['workerpoolPrice'], bar_width, 
                    label='Workerpool', color='#2ecc71')
    bars2 = plt.bar([i + bar_width for i in indices], period_sums['datasetPrice'], 
                    bar_width, label='Dataset', color='#3498db')
    bars3 = plt.bar([i + 2*bar_width for i in indices], period_sums['appPrice'], 
                    bar_width, label='App', color='#e74c3c')
    
    # Add value labels on top of bars
    def add_value_labels(bars):
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}',
                    ha='center', va='bottom')
    
    add_value_labels(bars1)
    add_value_labels(bars2)
    add_value_labels(bars3)
    
    # Customize the plot
    plt.xlabel('Time Period')
    plt.ylabel('Revenue')
    plt.title(f'{period.capitalize()} Revenue Distribution')
    plt.legend()
    
    # Format x-axis to show dates
    if period == 'daily':
        plt.xticks([i + bar_width for i in indices], 
                   period_sums['period'], rotation=90)
    else:
        plt.xticks([i + bar_width for i in indices], 
                   period_sums['period'], rotation=90)
    
    # Add grid for better readability
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    # Show the plot
    plt.show()
    
    return period_sums

# Function to plot all three time periods side by side
def plot_all_periods(df,show_monthly=True, show_weekly=True, show_daily=True):
    """
    Create three subplots showing daily, weekly, and monthly price distributions
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing DATE, workerpoolPrice, datasetPrice, and appPrice columns
    
    Returns:
    tuple: Daily, weekly, and monthly aggregated data
    """
    plt.figure(figsize=(35, 12))
    
    # Daily subplot
    if show_daily is True:
        daily_data = plot_price_histogram(df, 'daily')
      
    # Weekly subplot
    if show_weekly is True:
        weekly_data = plot_price_histogram(df, 'weekly')
   
    # Monthly subplot
    if show_monthly is True:
        monthly_data = plot_price_histogram(df, 'monthly')
    

plot_all_periods(df_latest, show_monthly=True, show_weekly=True, show_daily=True)

## Section 3 : Save dataset in google drive [USE CAREFULLY] 

Pre requisite : Google Drive connected
Save data in google Drive, Control filename to avoid overwrite existing dataset

In [3]:
# @title Save data in csv file on google drive [USE CAREFULLY!!!!!!]
from google.colab import drive
drive.mount('/content/drive')

#Define the period
date_begin = '2025-01-01 00:00:00'
date_end = '2025-02-01 00:00:00'

filename='/content/drive/Shareddrives/App_and_Product/dashboard_marketplace/data/TRD/TRD_2025_01.csv'
date_begin = datetime.strptime(date_begin, '%Y-%m-%d %H:%M:%S')
date_end = datetime.strptime(date_end, '%Y-%m-%d %H:%M:%S')

print ("Period to retrieve:", date_begin, date_end)

api_data = get_api_data_iexec_interval(url_query_bellecour, query, 0, date_begin, date_end)
df = pd.DataFrame(api_data, columns=["DATE", "workerpoolPrice", "datasetPrice","appPrice", "appName","appAddress", "workerpool"])
df["DATE"] = [datetime.utcfromtimestamp(int(x)) for x in df["DATE"]]

print("Period observed", df["DATE"].min(), df["DATE"].max())
print("dataset length: " + str(df.shape[0]))

# Save the DataFrame to CSV with the new filename
#df.to_csv(filename, index=False)


ModuleNotFoundError: No module named 'google'

## Section 4 : Load dataset from google drive


In [4]:
# @title Load dataset from csv file store on google drive

import os
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# Directory containing CSV files
directory='/content/drive/Shareddrives/App_and_Product/dashboard_marketplace/data/TRD/'

# List to store DataFrames from each CSV file
dfs = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Read CSV file into a DataFrame
        filepath = os.path.join(directory, filename)
        dftmp = pd.read_csv(filepath)
        dftmp['DATE'] = pd.to_datetime(dftmp['DATE'])
        _begin = dftmp['DATE'].min().strftime("%Y-%m-%d")
        _end = dftmp['DATE'].max().strftime("%Y-%m-%d")
        print(str(filename) + " " + str(_begin) + " " + str(_end))
        # Append DataFrame to the list
        dfs.append(dftmp)

# Concatenate all DataFrames into a single DataFrame
concatenated_df = pd.concat(dfs, ignore_index=True)


_begin = concatenated_df['DATE'].min().strftime("%Y-%m-%d")
_end = concatenated_df['DATE'].max().strftime("%Y-%m-%d")
print("global dataframe: " + str(concatenated_df.shape) +" " + str(_begin) + " " + str(_end))


ModuleNotFoundError: No module named 'google'

In [ ]:
# @title show header

concatenated_df

,TASK_ID,APP NAME,APP MULTIADDR,TAG,STATUS,DATE,WORKERPOOL ID,REQUESTER ID
0,0xfa4afe75ff9fb2c68cb0fc40e686e0d42c7bb1955eef...,python-hello-world:8.0.0-sconify-5.7.5-v14-pro...,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-01-31 23:57:35,0x0e7bc972c99187c191a17f3cae4a2711a4188c3f,0x6d6b5e88714263bbc6b4fa21a7328a7b947a302c
1,0x403050f30a97c01965687c5e90d86543b82a29b217e7...,python-hello-world:8.0.0-sconify-5.7.5-v14-debug,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-01-31 23:55:25,0xdb214a4a444d176e22030be1ed89da1b029320f2,0x7d695b9a362e15090ce26711808bc89c18efa70f
2,0x6feffc7cb97853cc0e4fd25c3e3fc4cf596e3bed0566...,tee-python-hello-world:8.0.3-gramine,docker.io/iexechub/tee-python-hello-world:8.0....,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-01-31 23:53:55,0x0e7bc972c99187c191a17f3cae4a2711a4188c3f,0x8d6f890ea5d10134cd6b9777457eb75e76312304
3,0x80e6c4810f9420aea0b82e24780e3c283a4215639253...,python-hello-world:7.0.5-sconify-5.3.15-v12-pr...,docker.io/iexechub/python-hello-world:7.0.5-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-01-31 23:53:25,0xeb14dc854a8873e419183c81a657d025ec70276b,0x1d6ce6c05043c28672218b103acb1e017babb68e
4,0xbdefde575ef5c7c9f470fb66b9e685ae961bfc18fd64...,python-hello-world:8.0.0-sconify-5.7.5-v14-pro...,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-01-31 23:37:25,0x0e7bc972c99187c191a17f3cae4a2711a4188c3f,0x6d6b5e88714263bbc6b4fa21a7328a7b947a302c
...,...,...,...,...,...,...,...,...
677440,0x4489abc89f1582bbde6b701637fbb90d702408c1733c...,python-hello-world:8.0.0-sconify-5.7.5-v14-pro...,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-02-01 00:17:40,0x0e7bc972c99187c191a17f3cae4a2711a4188c3f,0x6d6b5e88714263bbc6b4fa21a7328a7b947a302c
677441,0xf8646337e698b850dd9a901f6c593a4d53fcbb5a92d6...,python-hello-world:8.0.0-sconify-5.7.5-v14-debug,docker.io/iexechub/python-hello-world:8.0.0-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-02-01 00:15:35,0xdb214a4a444d176e22030be1ed89da1b029320f2,0x7d695b9a362e15090ce26711808bc89c18efa70f
677442,0xc1af2f8dc4dd2e4c46dda8d6f6d3e46a19398f480fd4...,tee-python-hello-world:8.0.3-gramine,docker.io/iexechub/tee-python-hello-world:8.0....,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-02-01 00:14:05,0x0e7bc972c99187c191a17f3cae4a2711a4188c3f,0x8d6f890ea5d10134cd6b9777457eb75e76312304
677443,0x300aa7505e364130eabd88dc77f1bc80b6e518801cfe...,python-hello-world:7.0.5-sconify-5.3.15-v12-pr...,docker.io/iexechub/python-hello-world:7.0.5-sc...,0x00000000000000000000000000000000000000000000...,COMPLETED,2024-02-01 00:13:05,0xeb14dc854a8873e419183c81a657d025ec70276b,0x1d6ce6c05043c28672218b103acb1e017babb68e


In [ ]:
# @title Display dataset struture and date info

print("Period observed", concatenated_df["DATE"].min(), concatenated_df["DATE"].max())

print("dataset length: " + str(concatenated_df.shape[0]))

concatenated_df.dtypes


Period observed 2020-07-15 15:00:15 2024-02-29 23:56:55
dataset length: 677445


TASK_ID                  object
APP NAME                 object
APP MULTIADDR            object
TAG                      object
STATUS                   object
DATE             datetime64[ns]
WORKERPOOL ID            object
REQUESTER ID             object
dtype: object

## Section 5 : Visualize full historical data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime


import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

def plot_price_histogram(df, period='daily'):
    """
    Create a histogram showing price sums by different time periods
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing DATE, workerpoolPrice, datasetPrice, and appPrice columns
    period (str): Aggregation period - 'daily', 'weekly', or 'monthly'
    
    Returns:
    pandas.DataFrame: Aggregated price data
    """
    # Ensure DATE is datetime
    df['DATE'] = pd.to_datetime(df['DATE'])
    
    # Create period groupings
    if period == 'daily':
        df['period'] = df['DATE'].dt.date
        date_format = '%Y-%m-%d'
    elif period == 'weekly':
        df['period'] = df['DATE'].dt.to_period('W').astype(str)
        date_format = 'Week %V, %Y'
    elif period == 'monthly':
        df['period'] = df['DATE'].dt.to_period('M').astype(str)
        date_format = '%B %Y'
    else:
        raise ValueError("Period must be 'daily', 'weekly', or 'monthly'")
    
    # Group by period and sum the prices
    period_sums = df.groupby('period').agg({
        'workerpoolPrice': 'sum',
        'datasetPrice': 'sum',
        'appPrice': 'sum'
    }).reset_index()
    
    if period == 'monthly':
      plt.figure(figsize=(20, 10))
    elif period == 'daily':
      plt.figure(figsize=(32, 10))
    elif period == 'weekly':
      plt.figure(figsize=(20, 10))


    # Plot each price type as a different colored bar
    bar_width = 0.25
    indices = range(len(period_sums['period']))
    
    bars1 = plt.bar(indices, period_sums['workerpoolPrice'], bar_width, 
                    label='Workerpool', color='#2ecc71')
    bars2 = plt.bar([i + bar_width for i in indices], period_sums['datasetPrice'], 
                    bar_width, label='Dataset', color='#3498db')
    bars3 = plt.bar([i + 2*bar_width for i in indices], period_sums['appPrice'], 
                    bar_width, label='App', color='#e74c3c')
    
    # Add value labels on top of bars
    def add_value_labels(bars):
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}',
                    ha='center', va='bottom')
    
    add_value_labels(bars1)
    add_value_labels(bars2)
    add_value_labels(bars3)
    
    # Customize the plot
    plt.xlabel('Time Period')
    plt.ylabel('Revenue')
    plt.title(f'{period.capitalize()} Revenue Distribution')
    plt.legend()
    
    # Format x-axis to show dates
    if period == 'daily':
        plt.xticks([i + bar_width for i in indices], 
                   period_sums['period'], rotation=90)
    else:
        plt.xticks([i + bar_width for i in indices], 
                   period_sums['period'], rotation=90)
    
    # Add grid for better readability
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    # Show the plot
    plt.show()
    
    return period_sums

# Function to plot all three time periods side by side
def plot_all_periods(df,show_monthly=True, show_weekly=True, show_daily=True):
    """
    Create three subplots showing daily, weekly, and monthly price distributions
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing DATE, workerpoolPrice, datasetPrice, and appPrice columns
    
    Returns:
    tuple: Daily, weekly, and monthly aggregated data
    """
    plt.figure(figsize=(35, 12))
    
    # Daily subplot
    if show_daily is True:
        daily_data = plot_price_histogram(df, 'daily')
      
    # Weekly subplot
    if show_weekly is True:
        weekly_data = plot_price_histogram(df, 'weekly')
   
    # Monthly subplot
    if show_monthly is True:
        monthly_data = plot_price_histogram(df, 'monthly')
    

plot_all_periods(concatenated_df, show_monthly=True, show_weekly=True, show_daily=False)

NameError: name 'concatenated_df' is not defined